I don't know why R has decided to be in French for me.

In [1]:
# install.packages("huge")
# install.packages("igraph")
# install.packages("GMPR")

In [2]:
install.packages("pscl")

Mise à jour de la liste HTML des packages dans '.Library'

Making 'packages.html' ...
 terminé



In [1]:
library(matrixStats)

In [2]:
# Taken from https://github.com/lichen-lab/GMPR/blob/master/GMPR.R
# as the library would not install correclty
GMPR <- function (comm, intersect.no = 10, ct.min = 1, trace = TRUE) {
	# Computes the GMPR size factor
	#
	# Args:
	#   comm: a matrix of counts, row - features (OTUs, genes, etc) , column - sample
	#   intersect.no: the minimum number of shared features between sample pair, where the ratio is calculated
	#   ct.min: the minimum number of counts required to calculate ratios

	#
	# Returns:
	#   a vector of the size factors with attribute 'NSS'. Samples with distinct sets of features will be output as NA.
	#         NSS:   number of samples with significant sharing (> intersect.no) including itself
	
	# mask counts < ct.min
	comm[comm < ct.min] <- 0
	
	if (is.null(colnames(comm))) {
		colnames(comm) <- paste0('S', 1:ncol(comm))
	}
	
	if (trace) cat('Begin GMPR size factor calculation ...\n')
	
	comm.no <- numeric(ncol(comm))
	gmpr <- sapply(1:ncol(comm),  function(i) {		
				if (i %% 50 == 0) {
					cat(i, '\n')
				}
				x <- comm[, i]
				# Compute the pairwise ratio
				pr <- x / comm
				# Handling of the NA, NaN, Inf
				pr[is.nan(pr) | !is.finite(pr) | pr == 0] <- NA
				# Counting the number of non-NA, NaN, Inf
				incl.no <- colSums(!is.na(pr))		
				# Calculate the median of PR
				pr.median <- colMedians(pr, na.rm=TRUE)
				# Record the number of samples used for calculating the GMPR
				comm.no[i] <<- sum(incl.no >= intersect.no)
				# Geometric mean of PR median
				if (comm.no[i] > 1) {
					return(exp(mean(log(pr.median[incl.no >= intersect.no]))))
				} else {
					return(NA)
				}
			}
	)
	
	if (sum(is.na(gmpr))) {
		warning(paste0('The following samples\n ', paste(colnames(comm)[is.na(gmpr)], collapse='\n'), 
				'\ndo not share at least ', intersect.no, ' common taxa with the rest samples! ',
				'For these samples, their size factors are set to be NA! \n', 
				'You may consider removing these samples since they are potentially outliers or negative controls!\n',
				'You may also consider decreasing the minimum number of intersecting taxa and rerun the procedure!\n'))
	}
	
	if (trace) cat('Completed!\n')
	if (trace) cat('Please watch for the samples with limited sharing with other samples based on NSS! They may be outliers! \n')
	names(gmpr) <- names(comm.no) <- colnames(comm)
	
	attr(gmpr, 'NSS') <- comm.no
	
	return(gmpr)
}

In [3]:
source("./ZiLN/inference.R")
source("./ZiLN/utils/utils.R")
source("./ZiLN/utils/magma_fix.R")
library("huge")
library("igraph")


Attachement du package : ‘igraph’


Les objets suivants sont masqués depuis ‘package:stats’:

    decompose, spectrum


L'objet suivant est masqué depuis ‘package:base’:

    union




In [4]:
load("./ZiLN/data/ll_deep.rda")
ls()

[1] "GMPR"                   "counts"                 "fit_parameters"        
 [4] "infer_Z"                "lamda_path"             "log_likelihood"        
 [7] "magma"                  "magma.norm"             "max_off_diagonal_value"
[10] "precision_recall"       "rmvzinegbin_new"        "taxmat"                
[13] "z_d"

In [5]:
# Get a boolean 1135x3957 matrix of whether the species
# was found in the person or not
nonzeros <- counts > 0

# Get the number of distinct people that possessed each species
num.nonzeros <- apply(nonzeros, 2, sum)

# Get the total amount of people
total.cells <- dim(counts)[1]

# Only keep the species who appear in more than 20% of the people
keep.indices <- (num.nonzeros / total.cells) > 0.2
counts_el <- as.matrix(counts[, keep.indices])
taxmat_el <- taxmat[keep.indices,]
kept_species <- colnames(counts[keep.indices])

First work out what the right value is.

In [8]:
# Get the zs
options(warn = -1) # turn warnings off because otherwise it's gonna scream...
zs <- infer_Z(counts_el)

# Get the matrix for ZiLN methodology
ziln.lambdas <- 10^seq(-0.1, -0.8, by=-0.01)
ziln.path <- huge(zs, lambda=ziln.lambdas)$path

Conducting Meinshausen & Buhlmann graph estimation (mb)....done


In [10]:
num.elements <- function(mat) {
    sum(mat != 0)
}

In [10]:
sapply(ziln.path, num.elements)[[36]]

[1] 1208

In [11]:
ziln.lambdas[[36]]

[1] 0.3548134

In [12]:
# Get the zs
options(warn = -1) # turn warnings off because otherwise it's gonna scream...
zs <- infer_Z(counts_el)

# Get the matrix for ZiLN methodology
result <- huge(zs, lambda=ziln.lambdas[[36]])$path[[1]]

Conducting Meinshausen & Buhlmann graph estimation (mb)....done


In [25]:
runtime <- 0
numtime <- 10
for (i in 1:numtime) {
    runtime <- runtime + system.time(huge(infer_Z(counts_el), lambda=ziln.lambdas[[36]])$path[[1]])[[1]]
}
runtime / numtime

Conducting Meinshausen & Buhlmann graph estimation (mb)....done
Conducting Meinshausen & Buhlmann graph estimation (mb)....done
Conducting Meinshausen & Buhlmann graph estimation (mb)....done
Conducting Meinshausen & Buhlmann graph estimation (mb)....done
Conducting Meinshausen & Buhlmann graph estimation (mb)....done
Conducting Meinshausen & Buhlmann graph estimation (mb)....done
Conducting Meinshausen & Buhlmann graph estimation (mb)....done
Conducting Meinshausen & Buhlmann graph estimation (mb)....done
Conducting Meinshausen & Buhlmann graph estimation (mb)....done
Conducting Meinshausen & Buhlmann graph estimation (mb)....done


[1] 1.6486

In [14]:
result

565 x 565 sparse Matrix of class "dgCMatrix"
                                                                               
  [1,] . 1 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
  [2,] 1 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
  [3,] . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
  [4,] . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
  [5,] . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
  [6,] . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
  [7,] . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
  [8,] . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
  [9,] . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
 [10,] . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ......
 [11,] . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . ..

In [14]:
get.assortativity.at.level <- function(adjacency.graph, taxmat, taxa.level) {
    groups <- as.integer(as.factor(taxmat[, taxa.level]))
    return(assortativity(adjacency.graph, groups))
}

In [16]:
get.assortativity.at.level(
    as.undirected(graph.adjacency(result)),
    taxmat_el,
    "V3" # phylum
)

[1] 0.1395338

In [17]:
get.assortativity.at.level(
    as.undirected(graph.adjacency(result)),
    taxmat_el,
    "V4" # class
)

[1] 0.1397643

In [18]:
get.assortativity.at.level(
    as.undirected(graph.adjacency(result)),
    taxmat_el,
    "V5" # order
)

[1] -0.0352612

Magma

In [30]:
magma.path <- magma(counts_el, method="mb")

Conducting copula median transformations....

Begin GMPR size factor calculation ...
50 
100 
150 
200 
250 
300 
350 
400 
450 
500 
550 
Completed!
Please watch for the samples with limited sharing with other samples based on NSS! They may be outliers! 
[1]  9.4902736 -0.2702236
[1] "magma fix"
[1]  9.4803643 -0.2986943
[1] "magma fix"
[1] 9.4628621 0.4502246
[1] "magma fix"
[1] 9.2984056 0.9219233
[1] "magma fix"
[1] 9.1428857 0.8278015
[1] "magma fix"
[1]  9.5534789 -0.2185147
[1] "magma fix"
[1] 10.6813419  0.7459852
[1] "magma fix"
[1]  8.377864 -1.414968
[1] "magma fix"
[1]  9.1736669 -0.9280545
[1] "magma fix"
[1]  7.802141 -1.771555
[1] "magma fix"
[1]  9.9577297 -0.1503469
[1] "magma fix"
[1] 11.803024  1.058861
[1] "magma fix"
[1]  9.181349 -0.653048
[1] "magma fix"
[1]  9.5812324 -0.3900399
[1] "magma fix"
[1]  9.3107770 -0.3273222
[1] "magma fix"
[1] 10.9873825 -0.1424229
[1] "magma fix"
[1]  9.730047 -1.584295
[1] "magma fix"
[1] 10.2947336  0.8464097
[1] "magma fix"
[1]  9.027595 -1.509333
[1] "magma fix"
[1]  7.748

In [31]:
summary(magma.path)

             Length Class     Mode     
method            1 -none-    character
path             10 -none-    list     
beta             10 -none-    list     
lambda           10 -none-    numeric  
sparsity         10 -none-    numeric  
df             5650 -none-    numeric  
sym               1 -none-    character
scr               1 -none-    logical  
cov.input         1 -none-    logical  
data         641275 -none-    numeric  
rawdata      641275 -none-    numeric  
param           565 -none-    list     
merge            10 -none-    list     
variability      10 -none-    numeric  
opt.index         1 -none-    numeric  
refit        319225 dgCMatrix S4       
opt.lambda        1 -none-    numeric  
opt.sparsity      1 -none-    numeric  
criterion         1 -none-    character

In [34]:
sapply(magma.path$path, num.elements)

[1]     0   182   454   942  1570  2520  3836  5592  7746 11196

In [35]:
get.assortativity.at.level(
    as.undirected(graph.adjacency(magma.path$path[[5]])),
    taxmat_el,
    "V3" # phylum
)

[1] 0.1223524

In [36]:
get.assortativity.at.level(
    as.undirected(graph.adjacency(magma.path$path[[5]])),
    taxmat_el,
    "V4" # class
)

[1] 0.09608017

In [37]:
get.assortativity.at.level(
    as.undirected(graph.adjacency(magma.path$path[[5]])),
    taxmat_el,
    "V5" # order
)

[1] 0.01167906

In [38]:
runtime <- 0
numtime <- 10
for (i in 1:numtime) {
    runtime <- runtime + system.time(magma(counts_el, method="mb"))[[1]]
}
runtime / numtime

Conducting copula median transformations....Begin GMPR size factor calculation ...
50 
100 
150 
200 
250 
300 
350 
400 
450 
500 
550 
Completed!
Please watch for the samples with limited sharing with other samples based on NSS! They may be outliers! 
[1]  9.4902736 -0.2702236
[1] "magma fix"
[1]  9.4803643 -0.2986943
[1] "magma fix"
[1] 9.4628621 0.4502246
[1] "magma fix"
[1] 9.2984056 0.9219233
[1] "magma fix"
[1] 9.1428857 0.8278015
[1] "magma fix"
[1]  9.5534789 -0.2185147
[1] "magma fix"
[1] 10.6813419  0.7459852
[1] "magma fix"
[1]  8.377864 -1.414968
[1] "magma fix"
[1]  9.1736669 -0.9280545
[1] "magma fix"
[1]  7.802141 -1.771555
[1] "magma fix"
[1]  9.9577297 -0.1503469
[1] "magma fix"
[1] 11.803024  1.058861
[1] "magma fix"
[1]  9.181349 -0.653048
[1] "magma fix"
[1]  9.5812324 -0.3900399
[1] "magma fix"
[1]  9.3107770 -0.3273222
[1] "magma fix"
[1] 10.9873825 -0.1424229
[1] "magma fix"
[1]  9.730047 -1.584295
[1] "magma fix"
[1] 10.2947336  0.8464097
[1] "magma fix"
[1]  9

[1] 22.1533